In [26]:
import os
from datetime import datetime
import concurrent
import pandas_datareader.data as web
import pandas as pd
import datetime
import concurrent.futures
from concurrent.futures import wait, ALL_COMPLETED
import yfinance as yf
import urllib
import urllib.request
import time
from datetime import timedelta

from ipywidgets import interactive
import numpy as np

from ipywidgets import interact, interactive, fixed, interact_manual
import ipywidgets as widgets

#frequency = output_variable.value
frequency = "Q"

w=117*8
#start = datetime.datetime(2010, 1, 1)

#end = datetime.datetime(2013, 1, 27)
end_date = datetime.date.today()

start_date = end_date - timedelta(weeks=w)

pd.set_option('display.max_columns', None) #replace n with the number of columns you want to see completely
pd.set_option('display.max_rows', None) #replace n with the number of rows you want to see completely

cores = int(len(os.sched_getaffinity(0)))

print(end_date)
print(start_date)


etf_commodities = ['DBO','CORN', 'WEAT', 'SOYB', 'JO', 'SGG', 'BAL', 'COW', 'MOO', 'TAGS', 'KOL' ]
#Gold, Silver, Platinum, Copper, Paladium, Aluminum, Iron, Steel
etf_metals = ['IAU', 'SLV', 'PGM', 'JJC', 'PALL', 'JJU', 'IFUNX', 'SLX']
#US dollar, European Euro, Japanese yen, Pound sterling, Australian dollar, Canadian dollar, Swiss franc, Chinese Yuan Renminbi, Swedish Krona, Peso, India
#defunct: Russia: XRU, Mexico: FXM
etf_foreign_exchanges = ['UUP','FXE','FXY','FXB','FXA','FXC','FXF','CYB', 'FXS', 'INR']
#residential, Ishares all NAmerica
etf_real_estate = ['REZ', 'IYR']
#Russia, Germany, UK, Japan, China, Euro, Euro, Brazil, Latin America, Mexico, India
etf_economies = ['ERUS','EWG','EWU','EWJ','MCHI','EZU','IEUR','EWZ','ILF','EWW','INDA']
#Ishares Investment Grade, IShares core aggregate Investment grade, Short, Total, 1-5 Years, 5-10 Years, 10 Years, Gov/Credit
#defunct:

etf_spdr_indexes = ['XLC','XLY','XLP','XLE','XLF','XLV','XLI','XLB','XLRE','XLK','XLU']
etf_dow_components = ['MMM','AXP','AMGN','AAPL','BA','CAT','CVX','CSCO','KO','DOW','GS','HD','HON','IBM','INTC','JNJ','JPM','MCD','MRK','MSFT','NKE','PG','CRM','TRV','UNH','VZ','V','WMT','WBA','DIS']

etf_bonds = ['LQD', 'AGG', 'NEAR', 'IUSB', 'ISTB', 'IMTB', 'ILTB', 'GBF']
etf_muni_bonds = ['MUB', 'SUB', 'MEAR']

etf_treasuries = ['AGZ', 'GOVT', 'BIL', 'SHV', 'SHY', 'IEI', 'IEF', 'TLT']

crypto = ['BTC-USD','ETH','RPL','BCH','EOS','LTC']

#,'GOLDAMGBD228NLBM',
FRED_Indicators = ['CPALTT01USQ657N','PAYEMS','IRLTLT01USM156N','MABMM301USM189S','LFWA64TTUSM647S','MANMM101USA189S','MICH','UMCSENT','CSCICP03USM665S','DGS10','DTB3','DGS3MO','CASTHPI','GDPC1','CIVPART','POPTOTUSA647NWDB','MEHOINUSA672N','HOSMEDUSM052N','MORTGAGE30US','TTLHH','CSUSHPINSA','EMRATIO','CPIAUCSL','PSAVERT','LRUN64TTUSQ156S','USSTHPI','NYSTHPI','M2V','GFDEBTN','DFII10','GFDEGDQ188S','CUSR0000SEHA','ETOTALUSQ176N','ERENTUSQ176N','RECPROUSM156N','T5YIFR','BAMLHYH0A0HYM2TRIV','BAMLCC0A1AAATRIV','GVZCLS','DGS1','BAMLCC0A4BBBTRIV','VXVCLS','IC4WSA','WILLMICROCAPPR','WILLLRGCAPVAL','CFNAIDIFF','MZMSL','KCFSI','T5YIE','TOTALSA','USSLIND','AWHAETP','CES0500000003','TCU','WTB3MS','WGS3MO','TWEXB','DEXCHUS','DEXUSUK','CILACBQ158SBOG','CES4348400001','FEDFUNDS','TDSP','PERMIT','GFDEGDQ188S','CP','PRFI','DRSFRMACBS','DRCCLACBS','DRBLACBS','DALLCIACBEP','USROA','USROE','RSAHORUSQ156S','MEFAINUSA672N','COMREPUSQ159N','HDTGPDUSQ163N','POP','NROU','FGCCSAQ027S','TEDRATE', 'VIXCLS', 'NFCI','INDPRO','LES1252881600Q','CUUR0000SEHA','LEU0252918500Q','BAA10Y','BAMLC0A0CM','BAMLH0A3HYC','BOGMBASE','DCOILBRENTEU','DCOILWTICO','DFF','DGS1MO','DGS30','DGS5','FPCPITOTLZGUSA','ICSA','INTDSRUSM193N','M1','M1V','MPRIME','PPIACO','SPCS20RSA','STLFSI2','T10Y2Y','T10Y3M','TB3MS','TREAST','UNRATE','WPU0911']
#FRED_Indicators = ["CPALTT01USQ657N","PAYEMS","IRLTLT01USM156N","MABMM301USM189S","LFWA64TTUSM647S","MANMM101USA189S","MICH","UMCSENT","CSCICP03USM665S","DGS10","DTB3","DGS3MO","CASTHPI","GDPC1","CIVPART","POPTOTUSA647NWDB","MEHOINUSA672N","HOSMEDUSM052N","MORTGAGE30US","TTLHH","CSUSHPINSA","EMRATIO","CPIAUCSL","PSAVERT","LRUN64TTUSQ156S","USSTHPI","NYSTHPI","M2V","GFDEBTN","DFII10","GFDEGDQ188S","CUSR0000SEHA","ETOTALUSQ176N","ERENTUSQ176N","RECPROUSM156N","T5YIFR","BAMLHYH0A0HYM2TRIV","BAMLCC0A1AAATRIV","GVZCLS","DGS1","BAMLCC0A4BBBTRIV","VXVCLS","IC4WSA","WILLMICROCAPPR","WILLLRGCAPVAL","CFNAIDIFF","MZMSL","KCFSI","T5YIE","TOTALSA","USSLIND","AWHAETP","CES0500000003","TCU","WTB3MS","WGS3MO","TWEXB","DEXCHUS","DEXUSUK","CILACBQ158SBOG","CES4348400001","FEDFUNDS","TDSP","PERMIT","GFDEGDQ188S","CP","PRFI","DRSFRMACBS","DRCCLACBS","DRBLACBS","DALLCIACBEP","USROA","USROE","RSAHORUSQ156S","MEFAINUSA672N","COMREPUSQ159N","HDTGPDUSQ163N","POP","NROU","FGCCSAQ027S","TEDRATE", "VIXCLS", "NFCI","INDPRO","LES1252881600Q","CUUR0000SEHA","LEU0252918500Q","BAA10Y","BAMLC0A0CM","BAMLH0A3HYC","BOGMBASE","DCOILBRENTEU","DCOILWTICO","DFF","DGS1MO","DGS30","DGS5","FPCPITOTLZGUSA","GOLDAMGBD228NLBM","ICSA","INTDSRUSM193N","M1","M1V","MPRIME","PPIACO","SPCS20RSA","STLFSI2","T10Y2Y","T10Y3M","TB3MS","TREAST","UNRATE","WPU0911"]

Indexes = ['^SP500TR', 'QQQ', 'DIA', 'VTWO']

etf_indexes_and_Crypto_list = [Indexes, etf_commodities, etf_metals, etf_foreign_exchanges, etf_real_estate, etf_economies, etf_bonds, etf_muni_bonds, etf_treasuries, crypto, etf_spdr_indexes, etf_dow_components]

commodities = []
for sublist in etf_indexes_and_Crypto_list:
    for val in sublist:
        commodities.append(val)

#pool2 = concurrent.futures.ProcessPoolExecutor(cores)

completed = []
def dl(name):
    subset = yf.download(name, start=start_date, end=end_date, auto_adjust=True).iloc[:, :6].dropna(axis=0, how='any')
    subset = subset[start_date.strftime('%Y-%m-%d'):end_date.strftime('%Y-%m-%d')]
    #sleep(4)
    if len(subset) != 0:
        completed.append(name)
        return (subset)

#futures2 = [pool2.submit(dl, args) for args in commodities]
#wait(futures2, timeout=4, return_when=ALL_COMPLETED)

futures2 = []
#pd.DataFrame()
for i in commodities:
    futures2.append(dl(i))

#print(futures2.describe())


2021-05-15
2003-06-07
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[***********

In [27]:

commodities_ = pd.DataFrame()

for x in range(0,len(completed)):
    values = futures2[x]
    values.index = pd.to_datetime(values.index)
    values = values.resample(frequency).mean().dropna()
    values['Symbol'] = x
    values = values.loc[~values.index.duplicated(keep='last')]
    values = values.reset_index()

    commodities_ = pd.concat([commodities_,values], axis=0)



In [28]:
commodities_pvt = pd.pivot_table(commodities_, values='Close', index=['Date'],columns=['Symbol'])
commodities_pvt.columns = completed
commodities_pvt.to_csv("/mnt/distvol/commodities.csv")


In [29]:
#commodities_pvt.describe()

In [30]:
def Fred_Data(name):
    temp = web.DataReader(str(name), 'fred', start_date, end_date)
    temp.index = pd.to_datetime(temp.index)
    temp = temp.resample(frequency).mean().dropna()
    return(temp)

#pool1 = concurrent.futures.ProcessPoolExecutor(cores)

#futures1 = [pool1.submit(Fred_Data, args) for args in FRED_Indicators]
#wait(futures1, timeout=None, return_when=ALL_COMPLETED)

FRED_set = []
FRED_completed = []
for i in FRED_Indicators:
    FRED_completed.append(i)
    FRED_set.append(Fred_Data(i))



In [31]:

FRED_pvt = pd.DataFrame()

for x in range(0,len(FRED_completed)):
    values = FRED_set[x]
    #values.index = pd.to_datetime(FRED_set["DATE"])
    values = values.resample(frequency).mean().dropna()
    #values['Symbol'] = x
    values = values.loc[~values.index.duplicated(keep='last')]
    #values = values.reset_index()

    FRED_pvt = pd.concat([FRED_pvt,values], axis=1)

#FRED_ = pd.pivot_table(commodities_, values='Close', index=['Date'],columns=['Symbol'])
#FRED_.to_csv("/mnt/distvol/FRED_set.csv")



In [32]:

#FRED_pvt = pd.pivot_table(FRED_, index=['DATE'])
FRED_pvt.to_csv("/mnt/distvol/FRED_pvt.csv")

#print(len(FRED_.columns))
#print(len(FRED_))
#FRED_pvt.columns = FRED_completed
#FRED_pvt.describe()

In [33]:
#FRED_pvt.set_index(FRED_pvt.index)

In [34]:
combined_set = pd.concat([FRED_pvt.set_index(FRED_pvt.index),commodities_pvt],axis=1)

In [ ]:
combined_set.describe().loc['count'].index[combined_set.describe().loc['count']<len(combined_set)]

In [46]:

drops = combined_set.describe().loc['count'].index[combined_set.describe().loc['count']<len(combined_set)]
print(drops)
filtered = combined_set.columns.tolist()

for d in drops:
    #print(d)
    filtered.remove(d)
#filtered.remove(drops.tolist())
#combined_set[filtered].to_csv("/mnt/distvol/combined_set.csv")

#drop first/last row
all_data = combined_set[filtered]
all_data = all_data.iloc[:-1,:]
all_data = all_data.iloc[1:,:]

all_data.to_csv("/mnt/distvol/combined_set.csv")
#filtered
all_data.describe()

Index(['CPALTT01USQ657N', 'PAYEMS', 'IRLTLT01USM156N', 'MABMM301USM189S',
       'LFWA64TTUSM647S', 'MANMM101USA189S', 'MICH', 'UMCSENT',
       'CSCICP03USM665S', 'CASTHPI',
       ...
       'SHV', 'IEI', 'BTC-USD', 'RPL', 'EOS', 'XLC', 'XLRE', 'DOW', 'CRM',
       'V'],
      dtype='object', length=135)


,DGS10,DTB3,DGS3MO,MORTGAGE30US,DFII10,T5YIFR,BAMLHYH0A0HYM2TRIV,BAMLCC0A1AAATRIV,DGS1,BAMLCC0A4BBBTRIV,IC4WSA,WILLMICROCAPPR,WILLLRGCAPVAL,T5YIE,WTB3MS,WGS3MO,DEXCHUS,DEXUSUK,TEDRATE,VIXCLS,NFCI,BAA10Y,BAMLC0A0CM,BAMLH0A3HYC,DCOILBRENTEU,DCOILWTICO,DFF,DGS1MO,DGS30,DGS5,ICSA,STLFSI2,T10Y2Y,T10Y3M,TREAST,^SP500TR,QQQ,DIA,IYR,EWG,EWU,EWJ,EZU,EWZ,ILF,EWW,LQD,SHY,IEF,TLT,ETH,BCH,LTC,XLY,XLP,XLE,XLF,XLV,XLI,XLB,XLK,XLU,MMM,AXP,AMGN,AAPL,BA,CAT,CVX,CSCO,KO,GS,HD,HON,IBM,INTC,JNJ,JPM,MCD,MRK,MSFT,NKE,PG,TRV,UNH,VZ,WMT,WBA,DIS
count,71.000000,71.000000,71.000000,71.000000,71.000000,71.000000,71.000000,71.000000,71.000000,71.000000,7.100000e+01,71.000000,71.000000,71.000000,71.000000,71.000000,71.000000,71.000000,71.000000,71.000000,71.000000,71.000000,71.000000,71.000000,71.000000,71.000000,71.000000,71.000000,71.000000,71.000000,7.100000e+01,71.000000,71.000000,71.000000,7.100000e+01,71.000000,71.000000,71.000000,71.000000,71.000000,71.000000,71.000000,71.000000,71.000000,71.000000,71.000000,71.000000,71.000000,71.000000,71.000000,71.000000,71.000000,71.000000,71.000000,71.000000,71.000000,71.000000,71.000000,71.000000,71.000000,71.000000,71.000000,71.000000,71.000000,71.000000,71.000000,71.000000,71.000000,71.000000,71.000000,71.000000,71.000000,71.000000,71.000000,71.000000,71.000000,71.000000,71.000000,71.000000,71.000000,71.000000,71.000000,71.000000,71.000000,71.000000,71.000000,71.000000,71.000000,71.000000
mean,2.947474,1.219017,1.245498,4.691375,0.905181,2.253311,882.739232,518.651236,1.450400,614.371530,4.027843e+05,9047.656684,50273.858603,1.831534,1.219785,1.246168,6.959828,1.583474,0.422681,18.911541,-0.343019,2.569049,1.617841,10.292331,70.761540,67.109926,1.332934,1.185688,3.626194,2.277965,4.032250e+05,-0.051949,1.296478,1.701976,1.672957e+06,3222.393513,88.144981,135.453332,50.634013,20.928104,25.073893,42.157400,30.189803,34.885700,26.790465,40.852212,84.532115,74.941191,82.865955,89.703788,18.533652,15.496930,22.547650,56.443478,32.017377,46.735013,17.315781,47.511856,40.520372,35.507203,36.731968,31.370554,99.458834,59.544478,96.783013,22.563484,116.179754,71.665541,67.317491,22.887512,28.035754,148.693506,80.680556,70.472617,102.367024,24.934682,72.993947,50.708834,81.713239,39.387186,50.214587,36.327940,60.608262,67.584609,97.917884,29.974364,59.533717,42.168991,61.517082
std,1.128088,1.510526,1.549367,1.068944,0.953879,0.337273,323.057435,118.418309,1.547402,189.564188,3.472624e+05,3240.531133,22465.655149,0.527435,1.509596,1.548441,0.700048,0.226844,0.383295,8.074154,0.577295,0.762256,0.945838,4.698193,26.489846,23.063863,1.623740,1.519340,1.038664,1.252790,3.184890e+05,1.124548,0.876513,1.115446,1.019771e+06,1673.431196,68.485686,69.021096,19.283531,5.995890,4.964083,9.444004,6.985486,13.221069,8.876023,12.814983,23.454719,7.412402,20.055862,30.852394,4.594984,7.863030,12.382468,37.045035,15.844490,13.688635,5.986690,27.390735,20.411835,14.123759,27.400762,14.454051,52.595364,28.274664,60.792139,27.305149,98.998938,38.985338,26.645424,10.890110,12.110248,50.458619,73.580572,50.021032,32.679644,13.689628,37.595782,31.205319,58.364719,19.696147,51.782754,31.819698,26.707566,38.556395,90.784530,14.075391,28.002706,17.911531,42.316143
min,0.650625,0.014754,0.014754,2.760714,-0.938594,1.483175,417.532687,347.606716,0.082459,353.016269,2.145385e+05,3470.198361,21713.933934,-0.501129,0.015385,0.015385,6.090100,1.232856,0.131967,10.307937,-0.777139,1.616613,0.835556,4.468750,28.426364,27.960476,0.058791,0.010164,1.362969,0.271719,2.136923e+05,-0.974269,-0.106935,-0.435000,4.749720e+05,1304.050655,26.048521,58.224690,18.182442,8.653265,13.544345,25.960220,15.551931,7.181351,6.297354,10.820426,51.586418,60.810088,51.049934,46.952564,7.379006,2.889494,3.755564,16.137871,13.062595,15.397239,5.830133,20.288994,15.233510,14.914910,12.337249,11.272887,36.272992,12.358085,34.007491,0.322440,23.513248,21.098690,18.966521,11.726162,12.314438,69.950669,16.103205,17.837421,49.814884,9.535503,30.855798,17.949507,13.811676,16.0165

,CPALTT01USQ657N,PAYEMS,IRLTLT01USM156N,MABMM301USM189S,LFWA64TTUSM647S,MANMM101USA189S,MICH,UMCSENT,CSCICP03USM665S,DGS10,DTB3,DGS3MO,CASTHPI,GDPC1,CIVPART,POPTOTUSA647NWDB,MEHOINUSA672N,HOSMEDUSM052N,MORTGAGE30US,TTLHH,CSUSHPINSA,EMRATIO,CPIAUCSL,PSAVERT,LRUN64TTUSQ156S,USSTHPI,NYSTHPI,M2V,GFDEBTN,DFII10,GFDEGDQ188S,CUSR0000SEHA,ETOTALUSQ176N,ERENTUSQ176N,RECPROUSM156N,T5YIFR,BAMLHYH0A0HYM2TRIV,BAMLCC0A1AAATRIV,GVZCLS,DGS1,BAMLCC0A4BBBTRIV,VXVCLS,IC4WSA,WILLMICROCAPPR,WILLLRGCAPVAL,CFNAIDIFF,MZMSL,KCFSI,T5YIE,TOTALSA,USSLIND,AWHAETP,CES0500000003,TCU,WTB3MS,WGS3MO,TWEXB,DEXCHUS,DEXUSUK,CILACBQ158SBOG,CES4348400001,FEDFUNDS,TDSP,PERMIT,GFDEGDQ188S,CP,PRFI,DRSFRMACBS,DRCCLACBS,DRBLACBS,DALLCIACBEP,USROA,USROE,RSAHORUSQ156S,MEFAINUSA672N,COMREPUSQ159N,HDTGPDUSQ163N,POP,NROU,FGCCSAQ027S,TEDRATE,VIXCLS,NFCI,INDPRO,LES1252881600Q,CUUR0000SEHA,LEU0252918500Q,BAA10Y,BAMLC0A0CM,BAMLH0A3HYC,BOGMBASE,DCOILBRENTEU,DCOILWTICO,DFF,DGS1MO,DGS30,DGS5,FPCPITOTLZGUSA,ICSA,INTDSRUSM193N,M1,M1V,MPRIME,PPIACO,SPCS20RSA,STLFSI2,T10Y2Y,T10Y3M,TB3MS,TREAST,UNRATE,WPU0911,^SP500TR,QQQ,DIA,VTWO,DBO,CORN,WEAT,SOYB,JO,SGG,BAL,COW,MOO,TAGS,KOL,IAU,SLV,PGM,JJC,PALL,JJU,IFUNX,SLX,UUP,FXE,FXY,FXB,FXA,FXC,FXF,CYB,FXS,INR,REZ,IYR,ERUS,EWG,EWU,EWJ,MCHI,EZU,IEUR,EWZ,ILF,EWW,INDA,LQD,AGG,NEAR,IUSB,ISTB,IMTB,ILTB,GBF,MUB,SUB,MEAR,AGZ,GOVT,BIL,SHV,SHY,IEI,IEF,TLT,BTC-USD,ETH,RPL,BCH,EOS,LTC,XLC,XLY,XLP,XLE,XLF,XLV,XLI,XLB,XLRE,XLK,XLU,MMM,AXP,AMGN,AAPL,BA,CAT,CVX,CSCO,KO,DOW,GS,HD,HON,IBM,INTC,JNJ,JPM,MCD,MRK,MSFT,NKE,PG,CRM,TRV,UNH,VZ,V,WMT,WBA,DIS
2003-09-30,0.490018,130198.000000,4.233333,6.072000e+12,1.873882e+08,NaN,2.333333,89.300000,100.383808,4.227656,0.928281,0.947813,386.94,13985.073,66.133333,NaN,NaN,NaN,6.033846,NaN,135.894333,62.066667,184.433333,5.766667,6.147594,287.56,467.15,1.905,6783320.0,2.203906,58.64208,206.233333,121682.0,3732.0,0.413333,2.268125,417.532687,347.606716,NaN,1.219219,353.016269,NaN,4.062885e+05,4772.894687,23335.269687,0.010000,6453.066667,-0.363806,1.779375,17.555000,1.183333,NaN,NaN,75.942767,0.927692,0.946154,118.993046,8.277167,1.610661,-12.0,1323.700000,1.016667,12.137832,1937.333333,58.64208,763.915,651.745,1.73,4.24,3.27,28845.0,1.39,15.17,68.3,NaN,NaN,NaN,291071.000000,5.077202,83311.0,0.201587,19.320156,-0.663105,93.752633,337.0,206.100000,958.0,2.575469,1.182239,11.168060,7.198667e+05,28.426364,30.221719,1.016848,0.918750,5.280625,3.138125,NaN,4.022308e+05,2.000000,1293.738462,8.941,4.000000,138.066667,143.643998,-0.005554,2.549375,3.279844,0.930000,6.537019e+05,6.133333,123.100000,1450.154060,28.081883,61.743101,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,21.670300,NaN,8.653265,13.544345,25.960220,NaN,15.551931,NaN,7.181351,6.297354,10.820426,NaN,51.586418,58.066826,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,60.810088,NaN,51.049934,46.952564,NaN,19.861081,NaN,2.889494,NaN,3.755564,NaN,22.253413,13.062595,15.397239,14.307710,21.294399,16.485220,14.914910,NaN,14.047095,11.272887,43.792416,30.142026,53.523079,0.322440,23.513248,21.098690,18.966521,14.214979,13.123854,NaN,69.950669,21.683826,17.837421,53.912464,16.302045,31.242078,21.196816,13.811676,27.587086,17.275098,5.572157,27.368460,NaN,22.987969,21.319325,13.911913,NaN,39.559052,22.193444,16.798414
2003-12-31,0.018060,130505.333333,4.286667,6.066467e+12,1.878763e+08,NaN,2.633333,91.966667,100.830828,4.285161,0.915645,0.935161,411.71,14145.645,66.033333,NaN,NaN,NaN,5.919286,NaN,138.789333,62.200000,185.133333,5.366667,5.938424,295.71,489.14,1.940,6997964.0,2.010806,59.45960,207.333333,122159.0,3833.0,0.453333,2.537419,441.456364,352.547879,NaN,1.296452,361.886515,NaN,3.711346e+05,5575.044219,25033.193281,0.110000,6415.366667,-0.550972,2.011290,17.136333,1.623333,NaN,NaN,76.730433,0.916923,0.936154,115.611114,8.276902,1.707939,-12.3,1331.766667,0.996667,12.213932,1972.333333,59.45960,816.612,683.107,1.76,4.43,2.85,24216.0,1.41,15.38,68.5,NaN,NaN,NaN,291796.333333,5.067059,82366.